In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, StringType

In [2]:
# ====================== INITIALISATION ======================
# Vérifiez si un SparkContext existe déjà
if 'sc' in locals() or 'sc' in globals():
    sc.stop()  # Arrêtez le SparkContext précédent
    
spark = SparkSession.builder \
    .appName("TrainALSModel") \
    .master("yarn") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 13:37:58 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
25/05/02 13:37:59 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
25/05/02 13:38:15 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [3]:
# ====================== CHARGEMENT DES DONNÉES ======================
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", StringType(), True),
])

movies_schema = StructType([
    StructField("movieId", IntegerType(), True),
    StructField("title", StringType(), True),
    StructField("genres", StringType(), True),
])

print("📥 Lecture des fichiers CSV depuis HDFS...")
ratings_df = spark.read.csv("hdfs:///input/rating.csv", header=True, schema=ratings_schema)
movies_df = spark.read.csv("hdfs:///input/movie.csv", header=True, schema=movies_schema)

📥 Lecture des fichiers CSV depuis HDFS...


In [4]:
# ====================== NETTOYAGE ======================
# Supprimer les lignes avec valeurs nulles
ratings_df = ratings_df.dropna(subset=["userId", "movieId", "rating"])

In [5]:
# ====================== DIVISION TRAIN / TEST ======================
train_df, test_df = ratings_df.randomSplit([0.8, 0.2], seed=42)

In [6]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# ====================== DÉFINIR ALS ======================
als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    nonnegative=True,
    coldStartStrategy="drop",
    implicitPrefs=False  # À ajuster si vous utilisez des données implicites
)

# ====================== GRID SEARCH ======================
param_grid = ParamGridBuilder() \
    .addGrid(als.rank, [8, 12, 16]) \
    .addGrid(als.maxIter, [10, 15, 20]) \
    .addGrid(als.regParam, [0.01, 0.05, 0.1]) \
    .build()

# ====================== VALIDATION CROISÉE ======================
crossval = CrossValidator(
    estimator=als,
    estimatorParamMaps=param_grid,
    evaluator=RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction"),
    numFolds=3,
    parallelism=4
)

# Lancer l'optimisation
cv_model = crossval.fit(train_df)

# Récupérer le meilleur modèle
best_model = cv_model.bestModel

print(f"🌟 Meilleurs paramètres :")
print(f"- Rank (facteurs latents) : {best_model.rank}")
print(f"- Itérations max : {best_model._java_obj.parent().getMaxIter()}")
print(f"- Régularisation (regParam) : {best_model._java_obj.parent().getRegParam()}")

# Prédictions sur le test set avec le meilleur modèle
best_predictions = best_model.transform(test_df)

🌟 Meilleurs paramètres :
- Rank (facteurs latents) : 16
- Itérations max : 20
- Régularisation (regParam) : 0.05


NameError: name 'evaluator' is not defined

In [7]:
# Prédictions sur le test set avec le meilleur modèle
best_predictions = best_model.transform(test_df)

# Définir l'évaluateur RMSE
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")

# Évaluer le RMSE final
final_rmse = evaluator.evaluate(best_predictions)
print(f"🚀 RMSE après optimisation : {final_rmse:.4f}")

[Stage 12832:==================>                                    (1 + 2) / 3]

🚀 RMSE après optimisation : 0.7847


In [ ]:
    # # ====================== ÉVALUATION ======================
    # print("📊 Évaluation du modèle...")
    # predictions = model.transform(test_df)

    # evaluator = RegressionEvaluator(
    #     metricName="rmse",
    #     labelCol="rating",
    #     predictionCol="prediction"
    # )
    # rmse = evaluator.evaluate(predictions)
    # print(f"✅ RMSE sur l'ensemble test : {rmse:.4f}")

In [ ]:
# # ====================== ENREGISTREMENT ======================
# print("💾 Sauvegarde du modèle dans HDFS (/models/als)...")
# model.write().overwrite().save("hdfs:///models/als")

# print("🎉 Modèle entraîné et sauvegardé avec succès.")

In [ ]:
spark.stop()